In [1]:
import time
import re
from bs4 import BeautifulSoup #use beautifulsoup to scrape data
from selenium import webdriver


url = "http://millercenter.org/the-presidency/presidential-speeches"

browser = webdriver.Chrome() #change this line depending on your system
browser.get(url) #go to the website 


#This is what we will change if we want the whole data. Right now it only runs for 44th president

for i in range(30,31):
    x = i
    absolutepath_president = f'/html/body/div[2]/div/main/div[2]/div/div[2]/article/div/div[2]/div/div/div/form/div[3]/fieldset/div/div/div/div[{x}]/label'
    president1 = browser.find_element_by_xpath(absolutepath_president)
    president1.click()
    time.sleep(2)




SCROLL_PAUSE_TIME = 0.8


#slight changes to be made!!

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(browser.page_source, 'lxml')


In [11]:
import urllib3 
import pandas as pd
from datetime import datetime
import time
import string
import numpy as np

link_str = []

for link in soup.findAll('a', href=True):
    if link != None:
        link_str.append(link['href'])

# now I want the link strings containting /the-presidency/
new_link_str = []

for element in link_str:
    if element.find("/presidential-speeches/") >= 0: 
        new_link_str.append(element)
        


http = urllib3.PoolManager()
list2=[]
president_names=[]
for element in new_link_str:
    url = "http://millercenter.org" + element
#     print(element)
    response = http.request('GET', url)
    soup_2 = BeautifulSoup(response.data, 'lxml')
    list2.append(soup_2.findAll('div', {"class":"transcript-inner"}))
    president_names.append(soup_2.findAll('div', {"class":"president-name"}))
    


#converting into string due to weird beautifulsoup output    
list3=[]
for i in range(len(list2)):
    if list2[i]:
        list3.append(str(list2[i]))
    else:
        
        #add comments for why I had to do it like this
        
        url = "http://millercenter.org" + new_link_str[i]
        response = http.request('GET', url)
        soup_2 = BeautifulSoup(response.data, 'lxml')
        list3.append(str(soup_2.findAll('div', {"class":"view-transcript"})))

# print(list3[1])

#this will be used to clean the text from html tags
def tag_cleanr(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

cleaned_speeches = []

#these replace lines could and should be replaced with regex since we covered that in the syllabus more extensively

for i in range(len(list3)):
    x = tag_cleanr(list3[i])
    x = x.replace('Transcript', '').replace('\r\n', '').replace('\n', '')
    cleaned_speeches.append(x)


#Get all the speech dates into a list

pattern = r'(\w+-\d+-\d{4})'
speech_dates=[]


for i in range(len(new_link_str)):
    match = re.findall(pattern, new_link_str[i])
    speech_dates.append(str(match))


for i in range(len(speech_dates)):
    #this feels like its terrible syntax but not sure if there is a nicer way, maybe I will replace it with regex
    
    speech_dates[i] = speech_dates[i].replace('[', '').replace(']', ''.replace("'", "")).replace("'", "")
    speech_dates[i] = datetime.strptime(speech_dates[i], '%B-%d-%Y')


df = pd.DataFrame({'dates':speech_dates, 'speeches':cleaned_speeches})

#this part is added because dataframe does not recognize [] as an empty input. 

df['speeches'].replace('[]', np.nan, inplace=True)

#now that I replaced [] with NaN, I can simply say drop all the rows with NaN elements

df.dropna(subset=['speeches'], inplace=True)




# for i in range(len(Speeches)): 
#     if df[df.nameindex[i]==[]:
#         df.drop(df.index[i])
        
print(df)




        dates                                           speeches
0  1932-12-06  [To the Senate and House of Representatives:In...
1  1932-11-05  [In these closing hours of the campaign I am c...
3  1932-10-21  [This campaign is more than a contest between ...
4  1932-08-11  [Mr. Chairman and my fellow citizens:In accept...
5  1932-05-31  [AN EMERGENCY has developed in the last few da...
6  1931-12-11  [THE PRESIDENT said:"In my recommendations to ...
7  1931-12-08  [To the Senate and House of Representatives:It...
8  1931-10-18  [My fellow citizens:This broadcast tonight mar...
9  1931-09-22  [GREAT BRITAIN'S DEPARTURE FROM THE GOLD STAND...
10 1931-06-21  [THE PRESIDENT made the following statement:Th...
11 1931-02-26  [To the House of Representatives:I return here...
12 1931-02-03  [THE PRESIDENT said:Certain Senators have issu...
13 1930-12-09  [THE PRESIDENT said:"I observe that measures h...
14 1930-12-02  [To the Senate and House of Representatives:I ...
15 1930-10-02  [Members o